# Input Distribution Analysis

We have prompt, rejected, chosen. All can be multiple times, only tuples <prompt, rejected, chosen> is "really" unique.

## Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
whole_df = pd.read_parquet('data_analysis/agent_prompt_cleaned.parquet')
train_df = pd.read_parquet('data_analysis/agent_train.parquet')
test_df = pd.read_parquet('data_analysis/agent_test.parquet')

In [ ]:
stripped_train_df = train_df[(train_df['chosen'].str.strip() != '')]
stripped_test_df = test_df[(test_df['chosen'].str.strip() != '')]

## Plot styling

In [ ]:
def style_wrapper(
        axis, title=None, x_label=None, y_label=None, show_legend=False,
        x_start_at_zero=False
):
    """
    Custom styling for matplotlib axis. Removes the upper and right spines, the remaining two
    have a thicker line. Adds horizontal grid lines. If x_start_at_zero is True, the x-axis starts at 0.
    """
    # Increase left and bottom spine thickness
    for spine in ['left', 'bottom']:
        axis.spines[spine].set_linewidth(2)

    # Remove top and right spine
    for spine in ['top', 'right']:
        axis.spines[spine].set_visible(False)

    # Increase axis tick thickness
    axis.tick_params(axis='both', width=2)

    # Add horizontal grid below data markers
    axis.yaxis.grid(True, linewidth=1)
    axis.set_axisbelow(True)

    # Set x-axis start at 0
    if x_start_at_zero:
        axis.set_xlim(left=0)

    # Set titles/axis labels
    if title:
        axis.set_title(title)
    if x_label:
        axis.set_xlabel(x_label)
    if y_label:
        axis.set_ylabel(y_label)

    # Toggle legend
    if show_legend:
        axis.legend()

## Dataset Size

In [ ]:
dfs = {'Combined Dataset': whole_df, 'Train/Eval Split': train_df, 'Test Split': test_df}

names = list(dfs.keys())
sizes = [len(df) for df in dfs.values()]

fig, ax = plt.subplots()
bars = ax.bar(names, sizes, color=['tab:blue', 'tab:orange', 'tab:green'])

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom')

style_wrapper(ax, title='Size of Data Splits', y_label='Number of Rows')

plt.tight_layout()
plt.show()

In [ ]:
total = len(dfs['Combined Dataset'])
percent_train = len(dfs['Train/Eval Split']) / total * 100
percent_test = len(dfs['Test Split']) / total * 100

# Plotting
_, ax = plt.subplots()
bars = ax.bar(['Train/Eval Split', 'Test Split'], [percent_train, percent_test],
              color=['tab:blue', 'tab:orange'])

# Add percentage labels
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom')

ax.set_ylim(0, 100)

style_wrapper(ax, title='Composition of Dataset', y_label='Percentage (%)')

plt.tight_layout()
plt.show()

The train/test split was performed on categories. From the raw dataset, 8 categories were randomly picked and extracted from the train dataset. This was performed in such a manner, that the resulting test dataset contains at least 1000 unique prompts. This way, the evaluation will be performed on previously unseen tasks, which should show how well the model generalizes.

## Prompt Count

In [ ]:
names = list(dfs.keys())
sizes = [df['prompt'].nunique() for df in dfs.values()]

fig, ax = plt.subplots()
bars = ax.bar(names, sizes, color=['tab:blue', 'tab:orange', 'tab:green'])

for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom')

style_wrapper(ax, title='Unique Prompts per Data Split', y_label='Number of Unique Prompts')

plt.tight_layout()
plt.show()

## Prompt Distribution

In [ ]:
def plot_prompt_distribution(df_list, title=None):
    """
    Shows a boxplot showing the distribution of unique chosen/rejected values per prompt.
    """

    prompt_counts = [
        df['prompt'].value_counts().values for df in df_list
    ]
    labels = ['Train Split', 'Test Split']

    _, plot_ax = plt.subplots()
    plot_ax.boxplot(prompt_counts, tick_labels=labels)
    style_wrapper(
        plot_ax, title=title, x_label=None, y_label='Frequency'
    )

    plt.tight_layout()
    plt.show()

In [ ]:
plot_prompt_distribution([train_df, test_df], title='Unique Prompts per Data Split')

## Chosen/Rejected distributions

In [ ]:
def plot_unique_response_distribution(df_list, title=None):
    """
    Shows a boxplot showing the distribution of unique chosen/rejected values per prompt.
    """

    # Calculate unique responses per prompt
    chosen_per_prompt = [
        df.groupby('prompt')['chosen'].nunique() for df in df_list
    ]
    labels = ['Train Split', 'Test Split']

    _, plot_ax = plt.subplots()
    plot_ax.boxplot(chosen_per_prompt, tick_labels=labels)
    style_wrapper(
        plot_ax, title=title, y_label='Responses per Prompt'
    )

    plt.tight_layout()
    plt.show()

In [ ]:
plot_unique_response_distribution([train_df, test_df], 'Unique Instances per Prompt per Data Split')

## Top Categories by Sample Count

In [ ]:
def truncate_labels(labels, max_length=20):
    # Truncate labels to a given max length
    return [label if len(label) <= max_length else label[:max_length - 3] + '...' for label in labels]


def plot_top_categories(df, top_n=15, title=None):
    category_counts = df['category'].value_counts()
    top_categories = category_counts.head(top_n)

    truncated_labels = truncate_labels(top_categories.index.tolist())
    _, plot_ax = plt.subplots()
    plot_ax.bar(truncated_labels, top_categories.values, color='tab:blue')

    style_wrapper(
        plot_ax, title=title,
        x_label='Category', y_label='Number of Samples',
    )

    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_top_categories(dfs['Train/Eval Split'], title='Top Categories by Sample Count in Train Split')

In [ ]:
plot_top_categories(dfs['Test Split'], title='Top Categories by Sample Count in Test Split')

## Samples per Category

In [ ]:
def plot_sample_count_boxplot(df_list, title=None):
    category_counts = [
        df['category'].value_counts().values for df in df_list
    ]
    labels = ['Train Split', 'Test Split']

    _, plot_ax = plt.subplots()
    plot_ax.boxplot(category_counts, tick_labels=labels)

    style_wrapper(
        plot_ax, title=title,
        y_label='Number of Samples per Category',
    )

    plt.tight_layout()
    plt.show()

In [ ]:
plot_sample_count_boxplot([train_df, test_df], title='Number of Samples per Category')

## Average Number of Prompts per Category

In [ ]:
def plot_prompts_per_category(df_list, title=None):
    grouped = [
        df.groupby('category')['prompt'].nunique().values for df in df_list
    ]
    labels = ['Train Split', 'Test Split']
    _, plot_ax = plt.subplots()
    plot_ax.boxplot(grouped, tick_labels=labels)

    style_wrapper(
        plot_ax, title=title,
        y_label='Number of Unique Prompts',
    )

    plt.tight_layout()
    plt.show()

In [ ]:
plot_prompts_per_category([train_df, test_df], title='Unique Prompts per Category')

## Tokens per Prompt/Chosen/Rejected

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('unsloth/Qwen2.5-3B-Instruct')

In [ ]:
for key in ['Train/Eval Split', 'Test Split']:
    df = dfs[key]
    for column in ['prompt', 'chosen', 'rejected']:
        df[column + '_tokens'] = df[column].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=False)))

In [ ]:
def plot_token_length_dist(df, title=None, clip_position=800):
    # Most prompts have sizes below 1000; only a few outliers go up to 8000
    df_filtered = df[(df['prompt_tokens'] <= clip_position) &
                     (df['chosen_tokens'] <= clip_position) &
                     (df['rejected_tokens'] <= clip_position)]

    _, plot_ax = plt.subplots()
    plot_ax.boxplot(
        df_filtered[['prompt_tokens', 'chosen_tokens', 'rejected_tokens']],
        tick_labels=['Prompt', 'Chosen', 'Rejected']
    )

    style_wrapper(
        plot_ax, title=title,
        y_label='Number of Prompts'
    )

    plt.tight_layout()
    plt.show()

In [ ]:
plot_token_length_dist(dfs['Train/Eval Split'], title='Tokens per Prompt in Train Split', clip_position=600)

In [ ]:
plot_token_length_dist(dfs['Test Split'], title='Tokens per Prompt in Test Split', clip_position=600)

## Worst Category Evaluation

In [ ]:
test_worst_cats = [
    'arithmetic', 'philosophy', 'social_iqa',
    'elementary_math_qa', 'biology', 'economics'
]

train_worst_cats = [
    'anachronisms', 'analogical_similarity', 'strange_stories', 'real_or_fake_text',
    'discourse_marker_prediction', 'contextual_parametric_knowledge_conflicts',
    'winowhy', 'swahili_english_proverbs', 'checkmate_in_one',
    'fantasy_reasoning', 'question_selection'
]

In [ ]:
train_best_cats = ['hindu_knowledge', 'hinglish_toxicity', 'intent_recognition', 'cause_and_effect', 'truthfulqa', 'language_identification']

In [ ]:
def get_cat_count(df, category_list=None):
    if category_list is not None:
        df = df[df['category'].isin(category_list)]
    return df['category'].value_counts()

In [ ]:
get_cat_count(stripped_train_df, train_best_cats)

In [ ]:
get_cat_count(train_df, train_best_cats)

In [ ]:
get_cat_count(stripped_test_df)

In [ ]:
train_df['category'].value_counts()

In [ ]:
def average_tokens_per_category(df, avg_tokenizer, category_list):
    """
    Computes the average number of tokens per category using a Hugging Face tokenizer.

    Parameters:
        :param df: DataFrame with category and text columns.
        :param avg_tokenizer: Hugging Face tokenizer.
        :param category_list:


    Returns:
        pd.Series: Average token count per category (sorted descending).
    """
    df = df.copy()
    df = df[df['category'].isin(category_list)]
    token_columns = ['orig_prompt', 'chosen', 'rejected']
    for token_column in token_columns:
        df.loc[:, f'{token_column}_token_count'] = df[token_column].apply(
            lambda x: len(avg_tokenizer.encode(x, add_special_tokens=False))
        )
    token_columns = [f'{col}_token_count' for col in token_columns]
    return (
        df.groupby('category')[token_columns]
        .mean()
        .sort_values(by=token_columns, ascending=False)
    )

In [ ]:
from transformers import AutoTokenizer

cat_tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-3B')

In [ ]:
average_tokens_per_category(train_df, cat_tokenizer, train_best_cats)

In [ ]:
train_df[train_df['category'] == 'misconceptions']

In [ ]:
train_df[train_df['category'] == 'checkmate_in_one']